FPCC2 - Laboratório 1

Professor: Fábio Morais

Aluno: Suelio Matias

21 de abril de 2024

In [67]:
import pandas as pd
import numpy as np
from scipy import stats

In [68]:
data_cursos = pd.read_csv('data/cursos-prouni.csv')
data_estados = pd.read_csv('data/estados.csv')

In [69]:
merged_data = pd.merge(data_cursos, data_estados, left_on=['uf_busca'], right_on=['subdivision'])

### Q1 - Qual a diferença entre as mensalidades médias e medianas dos cursos de Ciência da Computação das regiões Nordeste e Sudeste? Na sua opinião, essa diferença o que causa essa diferença? Ela é estatisticamente significativa? Justifique.

In [70]:
# Filtrando o DataFrame para cursos de Ciência da Computação nas regiões Nordeste e Sudeste
cc_ne_se = merged_data[(merged_data['curso_busca'] == 'Ciência da Computação') & 
                       (merged_data['region'].isin(['NE', 'SE']))]

# Calculando as mensalidades médias e medianas
mensalidade_stats = cc_ne_se.groupby('region')['mensalidade'].agg(['mean', 'median'])
mensalidade_stats['diff'] = mensalidade_stats['mean'] - mensalidade_stats['median']
mensalidade_stats

,mean,median,diff
region,,,
NE,712.734194,742.8,-30.065806
SE,899.059381,779.0,120.059381


Justificativa:

A diferença entre as mensalidades médias e medianas pode indicar uma distribuição de mensalidades que não é simétrica, o que sugere a presença de valores extremos (outliers). A média é mais sensível a outliers do que a mediana. No Nordeste, a diferença é maior, o que pode indicar uma distribuição mais dispersa das mensalidades.

Para determinar se essa diferença é estatisticamente significativa, podemos realizar um teste t-student para duas amostras independentes.

In [71]:
# Filtrando as mensalidades para as duas regiões
mensalidade_ne = cc_ne_se[cc_ne_se['region'] == 'NE']['mensalidade']
mensalidade_se = cc_ne_se[cc_ne_se['region'] == 'SE']['mensalidade']

# Teste t-student
t_stat, p_value = stats.ttest_ind(mensalidade_ne, mensalidade_se)

if p_value < 0.05:
    significance = 'A diferença é estatisticamente significativa (p < 0.05)'
else:
    significance = 'A diferença não é estatisticamente significativa (p >= 0.05)'

significance

'A diferença é estatisticamente significativa (p < 0.05)'

### Q2 - Quais são os top 10 cursos com maior valor de mensalidade pelo total de bolsas (diferentes tipos)? Existe uma relação entre o valor total de mensalidade e o total de bolsas por curso? Justifique.

In [72]:
# Calculando o total de bolsas
merged_data['total_bolsas'] = merged_data[['bolsa_integral_cotas', 'bolsa_integral_ampla', 
                                      'bolsa_parcial_cotas', 'bolsa_parcial_ampla']].sum(axis=1)

# Ordenando os cursos pela mensalidade total
top_10_cursos = merged_data.groupby('nome').agg({'mensalidade': 'mean', 'total_bolsas': 'sum'}).nlargest(10, 'mensalidade')
top_10_cursos


,mensalidade,total_bolsas
nome,,
Medicina,7168.341532,1177.0
Engenharia de Materiais e Nanotecnologia,4108.000000,2.0
Desenho Industrial,3467.000000,28.0
Sistemas Biomédicos,3420.000000,22.0
Comunicação Social - Cinema,2782.333333,7.0
Pilotagem Profissional de Aeronaves,2630.886000,44.0
Comunicação e Multimeios,2585.050000,4.0
Odontologia,2324.165149,2529.0
Música - Composição,2211.000000,1.0


Justificativa:

A média da mensalidade foi escolhida para considerar os cursos que possuem valores de mensalidades variadas. A soma total das bolsas foi escolhida para representar o total de bolsas disponíveis para cada curso. Para entender se há uma relação entre a mensalidade e o número total de bolsas podemos utilizar a correlação de Spearman.

In [73]:
correlation = merged_data['mensalidade'].corr(merged_data['total_bolsas'], method='spearman')
correlation


0.5115785923550468

Utilizando a correlação Spearman é retornado o valor de 0.5115 indicando uma relação moderada positiva.

### Q3 - Quais são os cursos tecnológicos do estado da Paraíba que oferecem mais bolsas de estudo e quantas bolsas de cada tipo estes cursos oferecem?

In [74]:
# Filtrando os cursos tecnológicos do estado da Paraíba
cursos_pb = merged_data.loc[(merged_data['grau'].str.contains('Tecnológico')) & 
                        (merged_data['uf_busca'] == 'PB')].copy()

# Calculando o total de bolsas
cursos_pb.loc[:, 'total_bolsas'] = cursos_pb[['bolsa_integral_cotas', 'bolsa_integral_ampla', 
                                        'bolsa_parcial_cotas', 'bolsa_parcial_ampla']].sum(axis=1)

# Ordenando os cursos pela quantidade de bolsas
cursos_pb = cursos_pb.groupby('nome')[
        ['total_bolsas', 'bolsa_integral_cotas', 'bolsa_integral_ampla', 'bolsa_parcial_cotas', 'bolsa_parcial_ampla']
    ].sum().nlargest(10, 'total_bolsas')

cursos_pb

,total_bolsas,bolsa_integral_cotas,bolsa_integral_ampla,bolsa_parcial_cotas,bolsa_parcial_ampla
nome,,,,,
Gestão de Recursos Humanos,124.0,31.0,20.0,3.0,70.0
Segurança no Trabalho,110.0,11.0,7.0,1.0,91.0
Redes de Computadores,106.0,16.0,10.0,0.0,80.0
Gestão da Tecnologia da Informação,99.0,12.0,6.0,1.0,80.0
Negócios Imobiliários,95.0,6.0,4.0,0.0,85.0
Logística,80.0,9.0,3.0,1.0,67.0
Design de Interiores,53.0,13.0,10.0,0.0,30.0
Radiologia,53.0,12.0,30.0,1.0,10.0
Gestão Comercial,27.0,14.0,5.0,5.0,3.0


### Q4 - Considerando os cursos de Ciência da Computação, Engenharia da Computação e Sistemas de Informação, qual o percentual de bolsas ofertadas para cada modalidade (a distância, integral, noturno, etc.) por cada um dos cursos? Existe uma modalidade que é predominante?

In [75]:
# Filtrando os cursos de Ciência da Computação, Engenharia da Computação e Sistemas de Informação
cursos_comp = merged_data[merged_data['curso_busca'].isin(['Ciência da Computação', 
                                                           'Engenharia de Computação', 
                                                           'Sistema de Informação'])]

turnos = ['Curso a Distância', 'Integral', 'Matutino', 'Noturno', 'Vespertino']

# Calculando o percentual de bolsas para cada turno por curso
for curso in cursos_comp['nome'].unique():
    print(f"Percentual de bolsas por turno para o curso {curso}:")
    curso_df = cursos_comp[cursos_comp['nome'] == curso]
    for turno in turnos:
        turno_df = curso_df[curso_df['turno'] == turno]
        total_bolsas_turno = turno_df['total_bolsas'].sum()
        percentual = (total_bolsas_turno / curso_df['total_bolsas'].sum()) * 100
        print(f"Turno: {turno.capitalize()} | Percentual de bolsas: {percentual:.2f}%")
    print('\n')

    


Percentual de bolsas por turno para o curso Engenharia de Computação:
Turno: Curso a distância | Percentual de bolsas: 11.87%
Turno: Integral | Percentual de bolsas: 3.87%
Turno: Matutino | Percentual de bolsas: 36.15%
Turno: Noturno | Percentual de bolsas: 46.49%
Turno: Vespertino | Percentual de bolsas: 1.62%


Percentual de bolsas por turno para o curso Ciência da Computação:
Turno: Curso a distância | Percentual de bolsas: 0.00%
Turno: Integral | Percentual de bolsas: 1.26%
Turno: Matutino | Percentual de bolsas: 24.15%
Turno: Noturno | Percentual de bolsas: 73.52%
Turno: Vespertino | Percentual de bolsas: 1.07%


Percentual de bolsas por turno para o curso Sistema de Informação:
Turno: Curso a distância | Percentual de bolsas: 0.00%
Turno: Integral | Percentual de bolsas: 0.00%
Turno: Matutino | Percentual de bolsas: 7.36%
Turno: Noturno | Percentual de bolsas: 92.04%
Turno: Vespertino | Percentual de bolsas: 0.60%




A modalidade "Noturno" é predominante nos três cursos